<a href="https://colab.research.google.com/github/architb1703/Toxic_Span/blob/archit/SSL_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install contractions

     |████████████████████████████████| 317kB 16.9MB/s 
     |████████████████████████████████| 245kB 28.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81696 sha256=2625ef26a9262e3c27015d0657f011640a7bd1e4254fad84d800ca36336d670b
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
path = '/content/drive/MyDrive/ToxicSpan_CS669V/BERT_Preprocess/civil.csv'

In [ ]:
data = pd.read_csv(path)

In [ ]:
data

In [ ]:
toxic_data = data[data['target']>=0.8]

In [ ]:
toxic_data = toxic_data['comment_text']

In [ ]:
toxic_data

4                       haha you guys are a bunch of losers.
31         Yet call out all Muslims for the acts of a few...
34         This bitch is nuts. Who would read a book by a...
289                                         You're an idiot.
306        Who cares!? Stark trek and Star Wars fans are ...
                                 ...                        
1804600    Wow. What entitled, arrogant , immature, loser...
1804712                                          Human scum.
1804713    This dirtbag is no Marine; he's a sadistic ter...
1804753    I saved where I could because my children matt...
1804825    Who is the jerk in the last row between the C ...
Name: comment_text, Length: 30831, dtype: object

In [ ]:
toxic_data.to_csv('/content/drive/MyDrive/ToxicSpan_CS669V/BERT_Preprocess/toxic_civil.csv', index=False)

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ToxicSpan_CS669V/BERT_Preprocess/toxic_civil.csv')

In [ ]:
from nltk.tokenize import TreebankWordTokenizer as twt

for i,string in enumerate(data['comment_text']):
  data['comment_text'][i] = twt().tokenize(string)

In [ ]:
#################### cleaning functions ####################
# emoji removing function
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

#convert accented text
import unicodedata
def convert_accented_chars(text):
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8', 'ignore')
    return text

#expand contraction
import contractions
import re 

# function to expand contractions
def expand_contractions(text):
    return(contractions.fix(text))
# function to remove digits
def remove_nums(text):
    return re.sub('[0-9]+', '', text)

# remove special chars
def remove_spe_chars(text):
    return re.sub('\.+', '', text)
#################### cleaning functions ####################    

In [ ]:
from nltk.tokenize import TreebankWordTokenizer as twt

for i,string in enumerate(data['comment_text']):
  for j,token in enumerate(data['comment_text'][i]):
    token = expand_contractions(token) #expand contraction
    token = remove_nums(token)#remove digits
    token = convert_accented_chars(token)#convert accented
    token = deEmojify(token)#remove emoji
    token = remove_spe_chars(token)#combine all multiple full stops occurring together
    data['comment_text'][i][j] = token

In [ ]:
for i,string in enumerate(data['comment_text']):
  data['comment_text'][i] = [x for x in data['comment_text'][i] if x!='']

In [ ]:
data.to_csv('/content/drive/MyDrive/ToxicSpan_CS669V/BERT_Preprocess/train_civil.csv')

In [ ]:
#Creating labels for semi-supervised learning

In [ ]:
!pip install transformers==2.6.0
!pip install seqeval
!pip install urllib3 --upgrade

     |████████████████████████████████| 133kB 6.1MB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.26.2
    Uninstalling urllib3-1.26.2:
      Successfully uninstalled urllib3-1.26.2


  Using cached https://files.pythonhosted.org/packages/f5/71/45d36a8df68f3ebb098d6861b2c017f3d094538c0fb98fa61d4dc43e69b9/urllib3-1.26.2-py2.py3-none-any.whl
ERROR: requests 2.23.0 has requirement urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you'll have urllib3 1.26.2 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.25.11
    Uninstalling urllib3-1.25.11:
      Successfully uninstalled urllib3-1.25.11


In [ ]:
import torch
from transformers import BertTokenizer, BertConfig, BertForTokenClassification, AdamW
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from tqdm import tqdm
device = 'cuda' if torch.cuda.is_available() else 'cpu' 

In [ ]:
model = torch.load('/content/drive/MyDrive/semi_sup-5.pt')
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/ToxicSpan_CS669V/BERT_Preprocess/train_civil.csv')

In [ ]:
#Class to take in tokenizer and model along with input data and calculate the span average f1 score
class Eval():
  def __init__(self, model, tokenizer, X, maxlen=500):
    self.model = model
    self.model.cuda()
    self.tokenizer = tokenizer
    self.maxlen = maxlen
    self.X = X
    self.prepare_data()

    self.X = torch.tensor(self.X)
    self.attention_mask = torch.tensor(self.attention_mask)

    data = TensorDataset(self.X, self.attention_mask)
    self.dataloader = DataLoader(data, batch_size=16, shuffle=False)
  
  def tokenize_data(self, x):
    x = [a.lstrip("'").rstrip("'") for a in x.rstrip("]").lstrip("[").split(', ')]
    sentence = []
    for i in range(len(x)):
      word = x[i]
      tokenized_word = self.tokenizer.tokenize(word)
      sentence.extend(tokenized_word)
    return(sentence)

  def get_attention_mask(self, x):
    return([[(i!=0) for i in text] for text in x])

  def prepare_data(self):
    for i in range(len(self.X)):
      self.X[i] = self.tokenize_data(self.X[i])
    self.X = pad_sequences([tokenizer.encode(text) for text in self.X], maxlen = self.maxlen, dtype='long', value=0.0, truncating='post', padding = 'post')
    self.attention_mask = self.get_attention_mask(self.X)
  
  def evaluate(self, flag):
    predictions = []
    X = []
    for batch in tqdm(self.dataloader):
      batch = tuple(t.to(device) for t in batch)
      b_input_id, b_input_mask = batch
      self.model.eval()
      
      with torch.no_grad():
        outputs = self.model(b_input_id, token_type_ids=None, attention_mask=b_input_mask)
      logits = outputs[0].detach().cpu().numpy()
      predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
      X.extend(b_input_id)
    self.pred_tags = [[p_i for p_i, l_i in zip(p, l) if l_i != 0][1:-1] for p, l in zip(predictions, X)]
    

In [ ]:
X = data['comment_text'].values

In [ ]:
pred = Eval(model, tokenizer, X[:10000])

In [ ]:
pred.evaluate(1)

100%|██████████| 625/625 [05:37<00:00,  1.85it/s]


In [ ]:
data['comment_text'][0]

In [ ]:
d = []

for i in range(len(pred.pred_tags)):
  d.append([data['comment_text'][i], [0]+pred.pred_tags[i]+[0]])

In [ ]:
d = pd.DataFrame(d, columns=['token_final', 'target_final'])

In [ ]:
d.to_pickle('/content/drive/MyDrive/ToxicSpan_CS669V/BERT_Preprocess/civil_train_new_7.pkl') 